In [42]:
from openai import OpenAI
from pinecone import Pinecone
import os
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv
from collections import defaultdict


In [12]:
pc = Pinecone(os.getenv("PINECONE_KEY"))
index = pc.Index("resume-index-full")

In [7]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [83]:
client = OpenAI(
    base_url='http://localhost:11434/v1',
    api_key='ollama'
)

In [102]:
def LLM_chatbot(user_messsage, developer_message):
    try:
        response_stream = client.chat.completions.create(
            model="mistral:latest",
            messages=[
                developer_message,
                user_messsage
            ],
            stream=True
        )

        for chunk in response_stream:
            if chunk.choices and len(chunk.choices) > 0:
                delta = chunk.choices[0].delta
                if delta and delta.content:
                    print(delta.content, end="", flush=True)

    except Exception as e:
        print(e)


In [116]:
from collections import defaultdict

def query_to_chat(query):

    embedded_query = model.encode(query).tolist()
    retrieved_chunks = index.query(
        vector=embedded_query,
        top_k=3, 
        include_metadata=True
    )

    prompt_header = (
        "You are a resume assistant chatbot for Aneesh Patne. Using the following information from the resume:\n\n"
    )


    dic = defaultdict(list)
    for match in retrieved_chunks.matches:
        section = match.metadata.get('section', 'Unknown Section')
        text = match.metadata.get('text', '')
        dic[section].append(text)


    chunks_text = ""
    for section, texts in dic.items():
        chunks_text += f"{section}:\n"
        for text in texts:
            chunks_text += f"{text}\n\n"

    prompt_footer = (
        f"Answer the following question based on all the provided information from the resume above:\n\"{query}\"\n\n"
        "Please provide a detailed, relevant, and professional response using all available context."
    )

  
    prompt = prompt_header + chunks_text + prompt_footer


    user_message = {"role": "user", "content": prompt}
    developer_message = {
        "role": "system",
        "content": (
            "You are a professional Resume Assistant bot that communicates on behalf of the candidate using Retrieval-Augmented Generation (RAG). "
            "Your task is to answer questions, provide clarifications, or engage in dialogue based solely on the resume data provided. "
            "Do not invent information that is not present in the resume. Thoroughly review all provided context to offer the best possible, "
            "detailed, and accurate answer. Maintain a formal yet approachable tone suitable for professional interactions."
        )
    }

    LLM_chatbot(user_message, developer_message)


In [117]:
query = 'Can you list All the Projects you have worked on?'
query_to_chat(query)

 Certainly! Here is a list of projects that Aneesh Patne has worked on:

1. Socio-Economic Impact of Pollution on Life Expectancy: This is an ongoing machine learning project that analyzes the impact of air pollution on life expectancy while considering multiple socio-economic parameters. Technologies used for this project include Google Vertex AI, SvelteKit, Express, and Machine Learning (not specified). The project started in January 2024 and is still active. Tagged as a machine learning, data analysis, and sustainability project.

2. Personal Chatbot: Aneesh developed an interactive chatbot during the period of March 2023 to June 2023. This personal chatbot was created using Python, GPT-3, Flask, and FAISS. The chatbot aims to provide accurate, context-specific replies based on Retrieval-Augmented Generation (RAG). Tagged as a project under AI, chatbot, language processing, and web development.